Imports

In [46]:
import pyodbc
import pandas as pd

Csatlakozzunk egy már létező adatbázishoz

In [47]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=master;"
    "Trusted_Connection=yes;",
    autocommit=True # nem kell mindig beírni, hogy conn.commit() minden execute után
)


cursor = conn.cursor()

cursor.execute("USE master")

Hozzuk lére az 'autosiskola' adatbázist

In [48]:
cursor.execute("IF EXISTS (SELECT name FROM sys.databases WHERE name = 'autosiskola') DROP DATABASE autosiskola")   ### újrafuttatás miatt
cursor.execute("CREATE DATABASE autosiskola")
cursor.execute("USE autosiskola")

conn.close()

Csatlakozás az 'autosiskola' adatbázishoz

In [49]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=autosiskola;"
    "Trusted_Connection=yes;",
    autocommit=True
)
cursor = conn.cursor()

Töröljük a táblákat, amennyiben léteznek (újrafuttatása miatt)

In [50]:
drop_tables_sql = """
DROP TABLE IF EXISTS Lessons;
DROP TABLE IF EXISTS Exams;
DROP TABLE IF EXISTS Vehicles;
DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Instructors;
"""

for table in drop_tables_sql.strip().split(";"):
    if table.strip():
        cursor.execute(table)


Hozzuk létre a táblákat

In [51]:
create_tables_sql = """
CREATE TABLE Students (
    tanulo_id INT PRIMARY KEY,
    nev NVARCHAR(100) NOT NULL,
    szuletesi_datum DATE NOT NULL,
    lakcim NVARCHAR(200) NOT NULL,
    telefonszam NVARCHAR(30),
    email NVARCHAR(100),
    jelentkezes_datum DATE NOT NULL
);

CREATE TABLE Instructors (
    oktato_id INT PRIMARY KEY,
    nev NVARCHAR(100) NOT NULL,
    jogositvany_tipus CHAR(1) NOT NULL CHECK (jogositvany_tipus IN ('A', 'B', 'C', 'D')),
    munkaviszony_kezdete DATE NOT NULL,
    telefonszam NVARCHAR(30),
    email NVARCHAR(100)
);

CREATE TABLE Lessons (
    ora_id INT PRIMARY KEY,
    tanulo_id INT NOT NULL,
    oktato_id INT NOT NULL,
    datum DATE NOT NULL,
    idopont TIME NOT NULL,
    ora_tipus NVARCHAR(20) NOT NULL CHECK (ora_tipus IN ('elméleti', 'gyakorlati')),
    helyszin NVARCHAR(100),
    FOREIGN KEY (tanulo_id) REFERENCES Students(tanulo_id),
    FOREIGN KEY (oktato_id) REFERENCES Instructors(oktato_id)
);

CREATE TABLE Vehicles (
    jarmu_id INT PRIMARY KEY,
    rendszam NVARCHAR(20) NOT NULL UNIQUE,
    marka NVARCHAR(50),
    tipus NVARCHAR(50),
    evjarat INT CHECK (evjarat >= 1990),
    muszaki_ervenyesseg DATE,
    oktato_id INT NOT NULL,
    FOREIGN KEY (oktato_id) REFERENCES Instructors(oktato_id)
);

CREATE TABLE Exams (
    vizsga_id INT PRIMARY KEY,
    tanulo_id INT NOT NULL,
    vizsga_tipus NVARCHAR(20) NOT NULL CHECK (vizsga_tipus IN ('elméleti', 'forgalmi')),
    datum DATE NOT NULL,
    eredmeny NVARCHAR(20) NOT NULL CHECK (eredmeny IN ('sikeres', 'sikertelen')),
    FOREIGN KEY (tanulo_id) REFERENCES Students(tanulo_id)
);
"""

# a cursor.execute egyszerre csak 1 utasítást tud végrehajtani, ezért végig kell iterálni a create_table_sql-en

for table in create_tables_sql.strip().split(";"):
    if table.strip():
        cursor.execute(table)



Táblák feltöltése adatokkal

In [52]:
### Fájlok beolvasása
students = pd.read_csv("students.csv", encoding="utf-8")
instructors = pd.read_csv("instructors.csv", encoding="utf-8")
lessons = pd.read_csv("lessons.csv", encoding="utf-8")
vehicles = pd.read_csv("vehicles.csv", encoding="utf-8")
exams = pd.read_csv("exams.csv", encoding="utf-8")


### Táblák feltöltées executemanyvel

students_insert_sql = """
INSERT INTO Students (tanulo_id, nev, szuletesi_datum, lakcim, telefonszam, email, jelentkezes_datum)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
students_adatok = students.values.tolist()
cursor.executemany(students_insert_sql, students_adatok)

instructors_insert_sql = """
INSERT INTO Instructors (oktato_id, nev, jogositvany_tipus, munkaviszony_kezdete, telefonszam, email)
VALUES (?, ?, ?, ?, ?, ?)
"""
instructors_adatok = instructors.values.tolist()
cursor.executemany(instructors_insert_sql, instructors_adatok)

lessons_insert_sql = """
INSERT INTO Lessons (ora_id, tanulo_id, oktato_id, datum, idopont, ora_tipus, helyszin)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
lessons_adatok = lessons.values.tolist()
cursor.executemany(lessons_insert_sql, lessons_adatok)

vehicles_insert_sql = """
INSERT INTO Vehicles (jarmu_id, rendszam, marka, tipus, evjarat, muszaki_ervenyesseg, oktato_id)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
vehicles_adatok = vehicles.values.tolist()
cursor.executemany(vehicles_insert_sql, vehicles_adatok)

exams_insert_sql = """
INSERT INTO Exams (vizsga_id, tanulo_id, vizsga_tipus, datum, eredmeny)
VALUES (?, ?, ?, ?, ?)
"""
exams_adatok = exams.values.tolist()
cursor.executemany(exams_insert_sql, exams_adatok)

7 lekérdezés és eredményeik (egyszerűbbektől az összetettekig)

In [53]:
sql_1 = """
SELECT nev, szuletesi_datum FROM Students ORDER BY szuletesi_datum
"""
cursor.execute(sql_1)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_1_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_1_df


,nev,szuletesi_datum
0,Kovács Julianna,1974-08-29
1,Kovács Pintér József,1975-10-08
2,Nagy Balogh Annamária,1977-08-05
3,Dr. Nagy Zoltánné,1978-10-17
4,Dr. Vargané Kis Edina,1978-11-14
5,Dr. Simon István Csaba,1978-11-28
6,Dr. Tamásné Takács Andrea Dóra,1980-11-21
7,Balogh Éva,1981-07-27
8,Farkas E. Margit,1986-12-13
9,Boros Tamásné,1988-06-18


In [54]:
sql_2 = """
SELECT nev FROM Students WHERE YEAR(szuletesi_datum) > 1990
"""
cursor.execute(sql_2)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_2_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_2_df


,nev
0,Kovács Molnár István
1,László József
2,E. Nagy Zsuzsanna
3,Tóth I. Hanna
4,Dr. Lakatos G. István
5,Farkas Sára
6,K. Nagy Magdolna
7,Dr. Lengyel Teréz


In [55]:
sql_3 = """
SELECT S.nev AS tanulo_nev, I.nev AS oktato_nev
FROM Lessons L
JOIN Students S ON L.tanulo_id = S.tanulo_id
JOIN Instructors I ON L.oktato_id = I.oktato_id
"""
cursor.execute(sql_3)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_3_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_3_df


,tanulo_nev,oktato_nev
0,Kovácsné Jónás Zsuzsanna,Dr. Fábiánné Rácz Margit
1,Tóth I. Hanna,Vörösné Horváth Edit Klára
2,Kovács Molnár István,Jakabné Balogh Magdolna Ilona
3,Kovács Julianna,Dr. Fábiánné Rácz Margit
4,Farkas Sára,Dr. Horváthné Török Etelka Alexandra
5,Dr. Lakatos G. István,Balogh Váradi Éva
6,Dr. Lengyel Teréz,E. Tóth Gergő
7,Dr. Lengyel Teréz,Vörösné Horváth Edit Klára
8,Farkas Sára,Dr. Fábiánné Rácz Margit
9,Nagy Balogh Annamária,E. Tóth Gergő


In [56]:
sql_4 = """
SELECT I.nev, COUNT(DISTINCT L.tanulo_id) AS tanulok_szama
FROM Lessons L
JOIN Instructors I ON L.oktato_id = I.oktato_id
GROUP BY I.nev
"""
cursor.execute(sql_4)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_4_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_4_df


,nev,tanulok_szama
0,Balogh Váradi Éva,1
1,Dr. Fábiánné Rácz Margit,3
2,Dr. Horváthné Török Etelka Alexandra,2
3,E. Tóth Gergő,4
4,Jakabné Balogh Magdolna Ilona,3
5,Mészáros Tóth Imre,1
6,Orosz Gyöngyi Julianna,1
7,Vörösné Horváth Edit Klára,4


In [57]:
sql_5 = """
SELECT I.nev, COUNT(DISTINCT L.tanulo_id) AS tanulok_szama
FROM Lessons L
JOIN Instructors I ON L.oktato_id = I.oktato_id
GROUP BY I.nev
HAVING COUNT(DISTINCT L.tanulo_id) >= 2
"""
cursor.execute(sql_5)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_5_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_5_df


,nev,tanulok_szama
0,Dr. Fábiánné Rácz Margit,3
1,Dr. Horváthné Török Etelka Alexandra,2
2,E. Tóth Gergő,4
3,Jakabné Balogh Magdolna Ilona,3
4,Vörösné Horváth Edit Klára,4


In [58]:
sql_6 = """
SELECT 
    CASE 
        WHEN GROUPING_ID(ora_tipus) = 1 THEN 'Összesen'
        ELSE ora_tipus
    END AS ora_tipus,
    COUNT(*) AS ora_db
FROM Lessons
GROUP BY ROLLUP (ora_tipus)
"""
cursor.execute(sql_6)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_6_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_6_df


,ora_tipus,ora_db
0,elméleti,10
1,gyakorlati,10
2,Összesen,20


In [59]:
sql_7 = """
SELECT 
    S.nev,
    S.szuletesi_datum,
    I.nev AS oktato_nev,
    RANK() OVER (
        PARTITION BY L.oktato_id 
        ORDER BY S.szuletesi_datum
    ) AS rang_szuletesi_sorrend
FROM Students S
JOIN Lessons L ON S.tanulo_id = L.tanulo_id
JOIN Instructors I ON L.oktato_id = I.oktato_id
"""
cursor.execute(sql_7)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_7_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_7_df


,nev,szuletesi_datum,oktato_nev,rang_szuletesi_sorrend
0,Boros Tamásné,1988-06-18,Vörösné Horváth Edit Klára,1
1,Dr. Lengyel Teréz,1996-03-03,Vörösné Horváth Edit Klára,2
2,Dr. Lakatos G. István,1999-01-30,Vörösné Horváth Edit Klára,3
3,Tóth I. Hanna,2000-10-10,Vörösné Horváth Edit Klára,4
4,Dr. Nagy Zoltánné,1978-10-17,Mészáros Tóth Imre,1
5,Kovács Julianna,1974-08-29,Dr. Fábiánné Rácz Margit,1
6,Kovácsné Jónás Zsuzsanna,1989-01-07,Dr. Fábiánné Rácz Margit,2
7,Farkas Sára,2003-06-27,Dr. Fábiánné Rácz Margit,3
8,Kovács Julianna,1974-08-29,Jakabné Balogh Magdolna Ilona,1
9,Boros Tamásné,1988-06-18,Jakabné Balogh Magdolna Ilona,2


7 CRUD művelet (egyszerűbbektől az összetettekig)

Create

In [60]:
sql_C1 = """
INSERT INTO Students (tanulo_id, nev, szuletesi_datum, lakcim, telefonszam, email, jelentkezes_datum)
VALUES (999, 'Teszt Elek', '2000-01-01', 'Teszt utca 1.', '0600000000', 'teszt@pelda.hu', '2025-01-01')
"""
cursor.execute(sql_C1)


Read

In [61]:
sql_C2 = "SELECT * FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C2)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_2_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_2_df


,tanulo_id,nev,szuletesi_datum,lakcim,telefonszam,email,jelentkezes_datum
0,999,Teszt Elek,2000-01-01,Teszt utca 1.,0600000000,teszt@pelda.hu,2025-01-01


Update

In [62]:
sql_C3 = """
UPDATE Students SET lakcim = 'Módosított utca 99.' WHERE tanulo_id = 999
"""
cursor.execute(sql_C3)

Read - Update-elt lakcím ellenörzése

In [63]:
sql_C4 = "SELECT nev, lakcim FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C4)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_4_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_4_df


,nev,lakcim
0,Teszt Elek,Módosított utca 99.


Delete

In [64]:
sql_C5 = "DELETE FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C5)


Read - Delete ellenőrzése

In [65]:
sql_C6 = "SELECT * FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C6)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_6_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_6_df


,tanulo_id,nev,szuletesi_datum,lakcim,telefonszam,email,jelentkezes_datum


In [66]:
sql_C7 = """
IF EXISTS (SELECT 1 FROM Students WHERE tanulo_id = 1)
   AND EXISTS (SELECT 1 FROM Instructors WHERE oktato_id = 1)
BEGIN
    INSERT INTO Lessons (ora_id, tanulo_id, oktato_id, datum, idopont, ora_tipus, helyszin)
    VALUES (9999, 1, 1, '2025-05-16', '08:00', 'gyakorlati', 'Központi gyakorlópálya')
END
"""
cursor.execute(sql_C7)


In [67]:
conn.close()